In [1]:
### PREPARATION ###

import pandas as pd
import numpy as np
from ta.momentum import RSIIndicator
from ta.trend import macd
from ta.volume import OnBalanceVolumeIndicator
from ta.volatility import AverageTrueRange
from ta.momentum import AwesomeOscillatorIndicator
from ta.momentum import KAMAIndicator
from ta.momentum import PercentagePriceOscillator
from ta.momentum import PercentageVolumeOscillator
from ta.momentum import ROCIndicator
from ta.momentum import StochRSIIndicator
from ta.momentum import TSIIndicator
from ta.momentum import WilliamsRIndicator
from ta.volume import AccDistIndexIndicator
from ta.volume import ChaikinMoneyFlowIndicator
from ta.volume import EaseOfMovementIndicator
from ta.volume import ForceIndexIndicator
from ta.volume import MFIIndicator
from ta.volume import NegativeVolumeIndexIndicator
from ta.volume import VolumePriceTrendIndicator
from ta.trend import ADXIndicator
from ta.trend import MassIndex
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import minmax_scale



#Loading Datasets

Axis = pd.read_csv (r'/home/kaustubh/Documents/Project1/Axis_Bank_BSE.csv')
n_50 = pd.read_csv (r'/home/kaustubh/Documents/Project1/NIFTY_50.csv')
n_bank = pd.read_csv (r'/home/kaustubh/Documents/Project1/NIFTY_BANK.csv')

#Renaming Columns and Dropping Date, Volume and Close Columns 

n_50.rename(columns = {'Date':'N50_Date', 'Open':'N50_Open', 'High':'N50_High', 'Low':'N50_Low', 'Close':'N50_Close', 'Adj Close': 'N50_Adj Close', 'Volume': 'N50_Volume'}, inplace = True)
n_bank.rename(columns = {'Date':'NB_Date','Open':'NB_Open', 'High':'NB_High', 'Low':'NB_Low', 'Close':'NB_Close', 'Adj Close': 'NB_Adj Close', 'Volume': 'NB_Volume'}, inplace = True)

Axis_Working = Axis.drop(['Adj Close'], axis = 1)
Nifty_50 = n_50.drop(['N50_Date', 'N50_Volume', 'N50_Close', 'N50_Open', 'N50_High', 'N50_Low', 'N50_Adj Close'], axis=1)
Nifty_Bank = n_bank.drop(['NB_Date', 'NB_Volume', 'NB_Close', 'NB_Open', 'NB_High', 'NB_Low', 'NB_Adj Close'], axis=1)

#Merging the datasets and deleting rows with any null values

Axis_Working = Axis_Working.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

Axis_Working.reset_index(drop=True, inplace=True)

#Axis_Working["Open"] = minmax_scale(Axis_Working["Open"], (0,1))
#Axis_Working["High"] = minmax_scale(Axis_Working["High"], (0,1))
#Axis_Working["Low"] = minmax_scale(Axis_Working["Low"], (0,1))
#Axis_Working["Close"] = minmax_scale(Axis_Working["Close"], (0,1))
#Axis_Working["Volume"] = minmax_scale(Axis_Working["Volume"], (0,1))

### FEATURE EXTRACTION ###

#Calculating Candle Body

Axis_Working["Candle Body"] = (Axis_Working["Close"] - Axis_Working["Open"])


#Calculating Upper and Lower Shadow lengths 

Axis_Working["Upper Shadow"] = ""
Axis_Working["Lower Shadow"] = ""

for i in range (len(Axis_Working)):
    if(Axis_Working.iat[i, 6] >= 0):
        Axis_Working.iat[i, 7] = Axis_Working.iat[i, 2] - Axis_Working.iat[i, 4]
    else:
        Axis_Working.iat[i, 7] = Axis_Working.iat[i, 2] - Axis_Working.iat[i, 1]
#Axis_Working["Upper Shadow"] = minmax_scale(Axis_Working["Upper Shadow"], (0,1))
        
for i in range (len(Axis_Working)):
    if(Axis_Working.iat[i, 6] >= 0):
        Axis_Working.iat[i, 8] = Axis_Working.iat[i, 1] - Axis_Working.iat[i, 3]
    else:
        Axis_Working.iat[i, 8] = Axis_Working.iat[i, 4] - Axis_Working.iat[i, 3]
#Axis_Working["Lower Shadow"] = minmax_scale(Axis_Working["Lower Shadow"], (0,1))

#Calculating Range

Axis_Working["Range"] = ""

for i in range(len(Axis_Working)):
    Axis_Working.iat[i, 9] = Axis_Working.iat[i, 2] - Axis_Working.iat[i, 3]
#Axis_Working["Range"] = minmax_scale(Axis_Working["Range"], (0,1))
                                                                        
#Determining the nature of Candle 

Axis_Working["Candle_Nature"] = ""

for i in range(len(Axis_Working)):
    if(Axis_Working.iat[i, 6] > 0):    #Bullish
        Axis_Working.iat[i, 10] = 1  
    else:                              #Bearish
        Axis_Working.iat[i, 10] = 0
        
#Determining the type of candle

Axis_Working["Candle Type"] = ""

for i in range(len(Axis_Working)):   
        
    #Marubozu
    if(0 <= Axis_Working.iat[i, 7]/Axis_Working.iat[i, 9] <= 0.15 and 0 <= Axis_Working.iat[i, 8]/Axis_Working.iat[i, 9] <= 0.15): 
        Axis_Working.iat[i, 11] = "Marubozu"
        
       
    #Spinning Top    
    if(0.1 < abs(Axis_Working.iat[i, 6])/Axis_Working.iat[i, 9] <= 0.3 and 0.35<=Axis_Working.iat[i, 7]/Axis_Working.iat[i, 9]<=0.45): 
        Axis_Working.iat[i, 11] = "Spinning Top"
        
        
    #Doji
    if(0 <= abs(Axis_Working.iat[i, 6])/Axis_Working.iat[i, 9] <= 0.05):
        Axis_Working.iat[i, 11] = "Doji"
        
    if(Axis_Working.iat[i, 8] != 0 and 0<=Axis_Working.iat[i, 7]/Axis_Working.iat[i, 9]<=0.3 and Axis_Working.iat[i, 8]/abs(Axis_Working.iat[i, 6]) >= 2):
        
        #Hammer
        if(Axis_Working.iat[i, 4] < Axis_Working.iat[i-1, 4] < Axis_Working.iat[i-2, 4] < Axis_Working.iat[i-3, 4]): 
            Axis_Working.iat[i, 11] = "Hammer"
            
        #Hanging Man
        if(Axis_Working.iat[i, 4] > Axis_Working.iat[i-1, 4] > Axis_Working.iat[i-2, 4] > Axis_Working.iat[i-3, 4]):
            Axis_Working.iat[i, 11] = "Hanging Man"
            
    if(Axis_Working.iat[i, 7] != 0 and 0<=Axis_Working.iat[i, 8]/Axis_Working.iat[i, 9]<=0.3 and Axis_Working.iat[i, 7]/abs(Axis_Working.iat[i, 6]) >= 2):
        
        #Shooting Star
        if(Axis_Working.iat[i, 4]> Axis_Working.iat[i-1, 4] > Axis_Working.iat[i-2, 4] > Axis_Working.iat[i-3, 4] ): 
            Axis_Working.iat[i, 11] = "Shooting Star"
            
        #Inverted Hammer
        if(Axis_Working.iat[i, 4]< Axis_Working.iat[i-1, 4] < Axis_Working.iat[i-2, 4] < Axis_Working.iat[i-3, 4]): 
            Axis_Working.iat[i, 11] = "Inverted Hammer"
            

Axis_Working["CB"] = abs(Axis_Working["Candle Body"]).copy()

Axis_Working["EMA"] = Axis_Working["CB"].ewm(span=25,adjust=False).mean()

for i in range(len(Axis_Working)):
    
    #Long
    if(i>=25 and Axis_Working.iat[i, 11] != "Marubozu" and Axis_Working.iat[i, 11] != "Spinning Top" and Axis_Working.iat[i, 11] != "Doji" and Axis_Working.iat[i, 11] != "Hammer" and Axis_Working.iat[i, 11] != "Hanging Man" and Axis_Working.iat[i, 11] != "Shooting Star" and Axis_Working.iat[i, 11] != "Inverted Hammer"):
        if(Axis_Working.iat[i, 12]/Axis_Working.iat[i, 13] >= 1):
                Axis_Working.iat[i, 11] = "Long"
    
    #Short
    if(i>=25 and Axis_Working.iat[i, 11] != "Marubozu" and Axis_Working.iat[i, 11] != "Spinning Top" and Axis_Working.iat[i, 11] != "Doji" and Axis_Working.iat[i, 11] != "Hammer" and Axis_Working.iat[i, 11] != "Hanging Man" and Axis_Working.iat[i, 11] != "Shooting Star" and Axis_Working.iat[i, 11] != "Inverted Hammer"):
        if(Axis_Working.iat[i, 12]/Axis_Working.iat[i, 13] <= 0.5):
                Axis_Working.iat[i, 11] = "Short"
    


        
Axis_Working = Axis_Working.drop(['CB', 'EMA'], axis = 1)
#Axis_Working["Candle Body"] = minmax_scale(Axis_Working["Candle Body"], (0,1))

flag_1 = 0
flag_2 = 0
gap_u = 0
gap_l = 0

for i in range(len(Axis_Working)):
    
    if(Axis_Working.iat[i, 4] < Axis_Working.iat[i-1, 4] < Axis_Working.iat[i-2, 4] < Axis_Working.iat[i-3, 4]):
        flag_1 = 1
    if(Axis_Working.iat[i, 4] > Axis_Working.iat[i-1, 4] > Axis_Working.iat[i-2, 4] > Axis_Working.iat[i-3, 4]):
        flag_2 = 1
    
    #Bullish Engulfing
    if(flag_1 == 1 and Axis_Working.iat[i, 10]== 1 and Axis_Working.iat[i-1, 10]== 0 and abs(Axis_Working.iat[i, 6])>abs(Axis_Working.iat[i-1, 6]) and Axis_Working.iat[i, 4] > Axis_Working.iat[i-1, 1] and Axis_Working.iat[i, 1] < Axis_Working.iat[i-1, 4]):
               Axis_Working.iat[i, 11] = "Bullish Engulfing"
    
    #Bearish Engulfing
    if(flag_2 == 1 and Axis_Working.iat[i, 10]== 0 and Axis_Working.iat[i-1, 10]== 1 and abs(Axis_Working.iat[i, 6])>abs(Axis_Working.iat[i-1, 6]) and Axis_Working.iat[i, 1] > Axis_Working.iat[i-1, 4] and Axis_Working.iat[i, 4] < Axis_Working.iat[i-1, 1]):
        Axis_Working.iat[i, 11] = "Bearish Engulfing"
    
    #Piercing
    if(flag_1 == 1 and Axis_Working.iat[i, 10]== 1 and Axis_Working.iat[i-1, 10]== 0 and Axis_Working.iat[i, 11] != "Bullish Engulfing"):
        if(Axis_Working.iat[i, 4]>Axis_Working.iat[i-1, 1] and Axis_Working.iat[i, 1]<Axis_Working.iat[i-1, 1]):
            gap_u = Axis_Working.iat[i, 4] - Axis_Working.iat[i-1, 1]
            diff = abs(Axis_Working.iat[i, 6]) - gap_u
            if(abs(diff)/abs(Axis_Working.iat[i, 6]) < 0.5 and abs(diff)/abs(Axis_Working.iat[i-1, 6]) > 0.5):
                   Axis_Working.iat[i, 11] = "Piercing"
                
        elif(Axis_Working.iat[i, 4]<Axis_Working.iat[i-1, 1] and Axis_Working.iat[i, 4]>Axis_Working.iat[i-1, 4]):
            gap_l = Axis_Working.iat[i-1, 4] - Axis_Working.iat[i, 1]
            diff = abs(Axis_Working.iat[i, 6]) - gap_l
            if(abs(diff)/abs(Axis_Working.iat[i, 6]) < 0.5 and abs(diff)/abs(Axis_Working.iat[i-1, 6]) > 0.5):
                    Axis_Working.iat[i, 11] = "Piercing"
                
    #Dark Cloud Cover           
    if(flag_2 == 1 and Axis_Working.iat[i, 10]== 0 and Axis_Working.iat[i-1, 10]== 1 and Axis_Working.iat[i, 11] != "Bearish Engulfing"):
        if(Axis_Working.iat[i, 1]>Axis_Working.iat[i-1, 4] and Axis_Working.iat[i, 4]<Axis_Working.iat[i-1, 4]):
            gap_u = Axis_Working.iat[i, 1] - Axis_Working.iat[i-1, 4]
            diff = abs(Axis_Working.iat[i, 6]) - gap_u
            if(abs(diff)/abs(Axis_Working.iat[i, 6]) < 0.5 and abs(diff)/abs(Axis_Working.iat[i-1, 6]) > 0.5):
                   Axis_Working.iat[i, 11] = "Dark Cloud Cover"
         
        elif(Axis_Working.iat[i, 1]<Axis_Working.iat[i-1, 4] and Axis_Working.iat[i, 1]>Axis_Working.iat[i-1, 1]):
            gap_l = Axis_Working.iat[i-1, 1] - Axis_Working.iat[i, 4]
            diff = abs(Axis_Working.iat[i, 6]) - gap_l
            if(abs(diff)/abs(Axis_Working.iat[i, 6]) < 0.5 and abs(diff)/abs(Axis_Working.iat[i-1, 6]) > 0.5):
                    Axis_Working.iat[i, 11] = "Dark Cloud Cover"
    
    #Bullish Harami
    if(flag_1 == 1 and Axis_Working.iat[i, 10]== 1 and Axis_Working.iat[i-1, 10]== 0 and abs(Axis_Working.iat[i-1, 6])>abs(Axis_Working.iat[i, 6]) and Axis_Working.iat[i, 4] < Axis_Working.iat[i-1, 1] and Axis_Working.iat[i, 1] > Axis_Working.iat[i-1, 4]):
                   Axis_Working.iat[i, 11] = "Bullish Harami"
    
    #Bearish Harami
    if(flag_2 == 1 and Axis_Working.iat[i, 10]== 0 and Axis_Working.iat[i-1, 10]== 1 and abs(Axis_Working.iat[i-1, 6])>abs(Axis_Working.iat[i, 6]) and Axis_Working.iat[i, 1] < Axis_Working.iat[i-1, 4] and Axis_Working.iat[i, 4] > Axis_Working.iat[i-1, 1]):
                    Axis_Working.iat[i, 11] = "Bearish Harami"
        
    
for i in range(len(Axis_Working)):
    
    #Morning Star
    if(Axis_Working.iat[i, 10] == 1 and Axis_Working.iat[i-2, 10] == 0 and Axis_Working.iat[i-2, 4] < Axis_Working.iat[i-3, 4] < Axis_Working.iat[i-4, 4] and Axis_Working.iat[i, 11] == "Long" and  (Axis_Working.iat[i-1, 11] == "Short" or Axis_Working.iat[i-1, 11] == "Doji" or Axis_Working.iat[i-1, 11] == "Spinning Top") and Axis_Working.iat[i-2, 11] == "Long"):
                   Axis_Working.iat[i, 11] = "Morning Star"
    
    #Evening Star
    if(Axis_Working.iat[i, 10] == 0 and Axis_Working.iat[i-2, 10] == 1 and Axis_Working.iat[i-2, 4] > Axis_Working.iat[i-3, 4] > Axis_Working.iat[i-4, 4] and Axis_Working.iat[i, 11] == "Long" and  (Axis_Working.iat[i-1, 11] == "Short" or Axis_Working.iat[i-1, 11] == "Doji" or Axis_Working.iat[i-1, 11] == "Spinning Top") and Axis_Working.iat[i-2, 11] == "Long"):
                Axis_Working.iat[i, 11] = "Evening Star"
        
    #Soldiers  
    if(Axis_Working.iat[i, 4] > Axis_Working.iat[i-1, 4] > Axis_Working.iat[i-2, 4] and Axis_Working.iat[i, 10] == 1 and Axis_Working.iat[i-1, 10] == 1 and Axis_Working.iat[i-2, 10] == 1 and Axis_Working.iat[i, 11] == "Long" and Axis_Working.iat[i-1, 11] == "Long" and Axis_Working.iat[i-2, 11] == "Long"):
               Axis_Working.iat[i, 11] = "Soldiers"
    
    #Crows
    if(Axis_Working.iat[i, 4] < Axis_Working.iat[i-1, 4] < Axis_Working.iat[i-2, 4] and Axis_Working.iat[i, 10] == 0 and Axis_Working.iat[i-1, 10] == 0 and Axis_Working.iat[i-2, 10] == 0 and Axis_Working.iat[i, 11] == "Long" and Axis_Working.iat[i-1, 11] == "Long" and Axis_Working.iat[i-2, 11] == "Long"):
            Axis_Working.iat[i, 11] = "Crows"
        
for i in range(len(Axis_Working)):
    #None
    if(Axis_Working.iat[i, 11] != "Marubozu" and Axis_Working.iat[i, 11] != "Spinning Top" and Axis_Working.iat[i, 11] != "Doji" and Axis_Working.iat[i, 11] != "Hammer" and Axis_Working.iat[i, 11] != "Hanging Man" and Axis_Working.iat[i, 11] != "Shooting Star" and Axis_Working.iat[i, 11] != "Inverted Hammer" and Axis_Working.iat[i, 11] != "Long" and Axis_Working.iat[i, 11] != "Short" and Axis_Working.iat[i, 11] != "Bullish Engulfing" and Axis_Working.iat[i, 11] != "Bearish Engulfing" and Axis_Working.iat[i, 11] != "Piercing" and Axis_Working.iat[i, 11] != "Dark Cloud Cover" and Axis_Working.iat[i, 11] != "Bullish Harami" and Axis_Working.iat[i, 11] != "Bearish Harami" and Axis_Working.iat[i, 11] != "Morning Star" and Axis_Working.iat[i, 11] != "Evening Star" and Axis_Working.iat[i, 11] != "Soldiers" and Axis_Working.iat[i, 11] != "Crows"): 
                          Axis_Working.iat[i, 11] = "None"
            
Axis_Working["Marubozu"] = ""
Axis_Working["Spinning Top"] = ""
Axis_Working["Doji"] = ""
Axis_Working["Hammer"] = ""
Axis_Working["Hanging Man"] = ""
Axis_Working["Shooting Star"] = ""
Axis_Working["Inverted Hammer"] = ""
Axis_Working["Long"] = ""
Axis_Working["Short"] = ""
Axis_Working["SBullish Engulfing"] = ""
Axis_Working["Beraish Engulfing"] = ""
Axis_Working["Piercing"] = ""
Axis_Working["Dark Cloud Cover"] = ""
Axis_Working["Bullish Harami"] = ""
Axis_Working["Bearish Harami"] = ""
Axis_Working["Morning Star"] = ""
Axis_Working["Evening Star"] = ""
Axis_Working["Soldiers"] = ""
Axis_Working["Crows"] =  ""
Axis_Working["None"] = ""


for i in range (len(Axis_Working)):
    
    if(Axis_Working.iat[i, 11] == "Marubozu"):
        Axis_Working.iat[i, 12] = 1
        Axis_Working.iat[i, 13] = 0
        Axis_Working.iat[i, 14] = 0
        Axis_Working.iat[i, 15] = 0
        Axis_Working.iat[i, 16] = 0
        Axis_Working.iat[i, 17] = 0
        Axis_Working.iat[i, 18] = 0
        Axis_Working.iat[i, 19] = 0
        Axis_Working.iat[i, 20] = 0
        Axis_Working.iat[i, 21] = 0
        Axis_Working.iat[i, 22] = 0
        Axis_Working.iat[i, 23] = 0
        Axis_Working.iat[i, 24] = 0
        Axis_Working.iat[i, 25] = 0
        Axis_Working.iat[i, 26] = 0
        Axis_Working.iat[i, 27] = 0
        Axis_Working.iat[i, 28] = 0
        Axis_Working.iat[i, 29] = 0
        Axis_Working.iat[i, 30] = 0
        Axis_Working.iat[i, 31] = 0
   
        
    if(Axis_Working.iat[i, 11] == "Bearish Marubozu"):
        Axis_Working.iat[i, 12] = 0
        Axis_Working.iat[i, 13] = 0
        Axis_Working.iat[i, 14] = 0
        Axis_Working.iat[i, 15] = 0
        Axis_Working.iat[i, 16] = 0
        Axis_Working.iat[i, 17] = 0
        Axis_Working.iat[i, 18] = 0
        Axis_Working.iat[i, 19] = 0
        Axis_Working.iat[i, 20] = 0
        Axis_Working.iat[i, 21] = 0
        Axis_Working.iat[i, 22] = 0
        Axis_Working.iat[i, 23] = 0
        Axis_Working.iat[i, 24] = 0
        Axis_Working.iat[i, 25] = 0
        Axis_Working.iat[i, 26] = 0
        Axis_Working.iat[i, 27] = 0
        Axis_Working.iat[i, 28] = 0
        Axis_Working.iat[i, 29] = 0
        Axis_Working.iat[i, 30] = 0
        Axis_Working.iat[i, 31] = 0
     
        
        
    if(Axis_Working.iat[i, 11] == "Spinning Top"):
        Axis_Working.iat[i, 12] = 0
        Axis_Working.iat[i, 13] = 1
        Axis_Working.iat[i, 14] = 0
        Axis_Working.iat[i, 15] = 0
        Axis_Working.iat[i, 16] = 0
        Axis_Working.iat[i, 17] = 0
        Axis_Working.iat[i, 18] = 0
        Axis_Working.iat[i, 19] = 0
        Axis_Working.iat[i, 20] = 0
        Axis_Working.iat[i, 21] = 0
        Axis_Working.iat[i, 22] = 0
        Axis_Working.iat[i, 23] = 0
        Axis_Working.iat[i, 24] = 0
        Axis_Working.iat[i, 25] = 0
        Axis_Working.iat[i, 26] = 0
        Axis_Working.iat[i, 27] = 0
        Axis_Working.iat[i, 28] = 0
        Axis_Working.iat[i, 29] = 0
        Axis_Working.iat[i, 30] = 0
        Axis_Working.iat[i, 31] = 0
      
        
    if(Axis_Working.iat[i, 11] == "Doji"):
        Axis_Working.iat[i, 12] = 0
        Axis_Working.iat[i, 13] = 0
        Axis_Working.iat[i, 14] = 1
        Axis_Working.iat[i, 15] = 0
        Axis_Working.iat[i, 16] = 0
        Axis_Working.iat[i, 17] = 0
        Axis_Working.iat[i, 18] = 0
        Axis_Working.iat[i, 19] = 0
        Axis_Working.iat[i, 20] = 0
        Axis_Working.iat[i, 21] = 0
        Axis_Working.iat[i, 22] = 0
        Axis_Working.iat[i, 23] = 0
        Axis_Working.iat[i, 24] = 0
        Axis_Working.iat[i, 25] = 0
        Axis_Working.iat[i, 26] = 0
        Axis_Working.iat[i, 27] = 0
        Axis_Working.iat[i, 28] = 0
        Axis_Working.iat[i, 29] = 0
        Axis_Working.iat[i, 30] = 0
        Axis_Working.iat[i, 31] = 0

        
    if(Axis_Working.iat[i, 11] == "Hammer"):
            Axis_Working.iat[i, 12] = 0
            Axis_Working.iat[i, 13] = 0
            Axis_Working.iat[i, 14] = 0
            Axis_Working.iat[i, 15] = 1
            Axis_Working.iat[i, 16] = 0
            Axis_Working.iat[i, 17] = 0
            Axis_Working.iat[i, 18] = 0
            Axis_Working.iat[i, 19] = 0
            Axis_Working.iat[i, 20] = 0
            Axis_Working.iat[i, 21] = 0
            Axis_Working.iat[i, 22] = 0
            Axis_Working.iat[i, 23] = 0
            Axis_Working.iat[i, 24] = 0
            Axis_Working.iat[i, 25] = 0
            Axis_Working.iat[i, 26] = 0
            Axis_Working.iat[i, 27] = 0
            Axis_Working.iat[i, 28] = 0
            Axis_Working.iat[i, 29] = 0
            Axis_Working.iat[i, 30] = 0
            Axis_Working.iat[i, 31] = 0
         
            
    if(Axis_Working.iat[i, 11] == "Hanging Man"):
            Axis_Working.iat[i, 12] = 0
            Axis_Working.iat[i, 13] = 0
            Axis_Working.iat[i, 14] = 0
            Axis_Working.iat[i, 15] = 0
            Axis_Working.iat[i, 16] = 1
            Axis_Working.iat[i, 17] = 0
            Axis_Working.iat[i, 18] = 0
            Axis_Working.iat[i, 19] = 0
            Axis_Working.iat[i, 20] = 0
            Axis_Working.iat[i, 21] = 0
            Axis_Working.iat[i, 22] = 0
            Axis_Working.iat[i, 23] = 0
            Axis_Working.iat[i, 24] = 0
            Axis_Working.iat[i, 25] = 0
            Axis_Working.iat[i, 26] = 0
            Axis_Working.iat[i, 27] = 0
            Axis_Working.iat[i, 28] = 0
            Axis_Working.iat[i, 29] = 0
            Axis_Working.iat[i, 30] = 0
            Axis_Working.iat[i, 31] = 0
           
            
    
    if(Axis_Working.iat[i, 11] == "Shooting Star"):
            Axis_Working.iat[i, 12] = 0
            Axis_Working.iat[i, 13] = 0
            Axis_Working.iat[i, 14] = 0
            Axis_Working.iat[i, 15] = 0
            Axis_Working.iat[i, 16] = 0
            Axis_Working.iat[i, 17] = 1
            Axis_Working.iat[i, 18] = 0
            Axis_Working.iat[i, 19] = 0
            Axis_Working.iat[i, 20] = 0
            Axis_Working.iat[i, 21] = 0
            Axis_Working.iat[i, 22] = 0
            Axis_Working.iat[i, 23] = 0
            Axis_Working.iat[i, 24] = 0
            Axis_Working.iat[i, 25] = 0
            Axis_Working.iat[i, 26] = 0
            Axis_Working.iat[i, 27] = 0
            Axis_Working.iat[i, 28] = 0
            Axis_Working.iat[i, 29] = 0
            Axis_Working.iat[i, 30] = 0
            Axis_Working.iat[i, 31] = 0 
       
            
    if(Axis_Working.iat[i, 11] == "Inverted Hammer"):
            Axis_Working.iat[i, 12] = 0
            Axis_Working.iat[i, 13] = 0
            Axis_Working.iat[i, 14] = 0
            Axis_Working.iat[i, 15] = 0
            Axis_Working.iat[i, 16] = 0
            Axis_Working.iat[i, 17] = 0
            Axis_Working.iat[i, 18] = 1
            Axis_Working.iat[i, 19] = 0
            Axis_Working.iat[i, 20] = 0
            Axis_Working.iat[i, 21] = 0
            Axis_Working.iat[i, 22] = 0
            Axis_Working.iat[i, 23] = 0
            Axis_Working.iat[i, 24] = 0
            Axis_Working.iat[i, 25] = 0
            Axis_Working.iat[i, 26] = 0
            Axis_Working.iat[i, 27] = 0
            Axis_Working.iat[i, 28] = 0
            Axis_Working.iat[i, 29] = 0
            Axis_Working.iat[i, 30] = 0
            Axis_Working.iat[i, 31] = 0 
            
            
    if(Axis_Working.iat[i, 11] == "Long"):
            Axis_Working.iat[i, 12] = 0
            Axis_Working.iat[i, 13] = 0
            Axis_Working.iat[i, 14] = 0
            Axis_Working.iat[i, 15] = 0
            Axis_Working.iat[i, 16] = 0
            Axis_Working.iat[i, 17] = 0
            Axis_Working.iat[i, 18] = 0
            Axis_Working.iat[i, 19] = 1
            Axis_Working.iat[i, 20] = 0
            Axis_Working.iat[i, 21] = 0
            Axis_Working.iat[i, 22] = 0
            Axis_Working.iat[i, 23] = 0
            Axis_Working.iat[i, 24] = 0
            Axis_Working.iat[i, 25] = 0
            Axis_Working.iat[i, 26] = 0
            Axis_Working.iat[i, 27] = 0
            Axis_Working.iat[i, 28] = 0
            Axis_Working.iat[i, 29] = 0
            Axis_Working.iat[i, 30] = 0
            Axis_Working.iat[i, 31] = 1
     
            
    if(Axis_Working.iat[i, 11] == "Short"):
            Axis_Working.iat[i, 12] = 0
            Axis_Working.iat[i, 13] = 0
            Axis_Working.iat[i, 14] = 0
            Axis_Working.iat[i, 15] = 0
            Axis_Working.iat[i, 16] = 0
            Axis_Working.iat[i, 17] = 0
            Axis_Working.iat[i, 18] = 0
            Axis_Working.iat[i, 19] = 0
            Axis_Working.iat[i, 20] = 1
            Axis_Working.iat[i, 21] = 0
            Axis_Working.iat[i, 22] = 0
            Axis_Working.iat[i, 23] = 0
            Axis_Working.iat[i, 24] = 0
            Axis_Working.iat[i, 25] = 0
            Axis_Working.iat[i, 26] = 0
            Axis_Working.iat[i, 27] = 0
            Axis_Working.iat[i, 28] = 0
            Axis_Working.iat[i, 29] = 0
            Axis_Working.iat[i, 30] = 0
            Axis_Working.iat[i, 31] = 1
        
            
    if(Axis_Working.iat[i, 11] == "Bullish Engulfing"):
            Axis_Working.iat[i, 12] = 0
            Axis_Working.iat[i, 13] = 0
            Axis_Working.iat[i, 14] = 0
            Axis_Working.iat[i, 15] = 0
            Axis_Working.iat[i, 16] = 0
            Axis_Working.iat[i, 17] = 0
            Axis_Working.iat[i, 18] = 0
            Axis_Working.iat[i, 19] = 0
            Axis_Working.iat[i, 20] = 0
            Axis_Working.iat[i, 21] = 1
            Axis_Working.iat[i, 22] = 0
            Axis_Working.iat[i, 23] = 0
            Axis_Working.iat[i, 24] = 0
            Axis_Working.iat[i, 25] = 0
            Axis_Working.iat[i, 26] = 0
            Axis_Working.iat[i, 27] = 0
            Axis_Working.iat[i, 28] = 0
            Axis_Working.iat[i, 29] = 0
            Axis_Working.iat[i, 30] = 0
            Axis_Working.iat[i, 31] = 0
         
            
    if(Axis_Working.iat[i, 11] == "Bearish Engulfing"):
            Axis_Working.iat[i, 12] = 0
            Axis_Working.iat[i, 13] = 0
            Axis_Working.iat[i, 14] = 0
            Axis_Working.iat[i, 15] = 0
            Axis_Working.iat[i, 16] = 0
            Axis_Working.iat[i, 17] = 0
            Axis_Working.iat[i, 18] = 0
            Axis_Working.iat[i, 19] = 0
            Axis_Working.iat[i, 20] = 0
            Axis_Working.iat[i, 21] = 0
            Axis_Working.iat[i, 22] = 1
            Axis_Working.iat[i, 23] = 0
            Axis_Working.iat[i, 24] = 0
            Axis_Working.iat[i, 25] = 0
            Axis_Working.iat[i, 26] = 0
            Axis_Working.iat[i, 27] = 0
            Axis_Working.iat[i, 28] = 0
            Axis_Working.iat[i, 29] = 0
            Axis_Working.iat[i, 30] = 0
            Axis_Working.iat[i, 31] = 0
         
            
    if(Axis_Working.iat[i, 11] == "Piercing"):
                Axis_Working.iat[i, 12] = 0
                Axis_Working.iat[i, 13] = 0
                Axis_Working.iat[i, 14] = 0
                Axis_Working.iat[i, 15] = 0
                Axis_Working.iat[i, 16] = 0
                Axis_Working.iat[i, 17] = 0
                Axis_Working.iat[i, 18] = 0
                Axis_Working.iat[i, 19] = 0
                Axis_Working.iat[i, 20] = 0
                Axis_Working.iat[i, 21] = 0
                Axis_Working.iat[i, 22] = 0
                Axis_Working.iat[i, 23] = 1
                Axis_Working.iat[i, 24] = 0
                Axis_Working.iat[i, 25] = 0
                Axis_Working.iat[i, 26] = 0
                Axis_Working.iat[i, 27] = 0
                Axis_Working.iat[i, 28] = 0
                Axis_Working.iat[i, 29] = 0
                Axis_Working.iat[i, 30] = 0
                Axis_Working.iat[i, 31] = 0
               
                
    if(Axis_Working.iat[i, 11] == "Dark Cloud Cover"):
                Axis_Working.iat[i, 12] = 0
                Axis_Working.iat[i, 13] = 0
                Axis_Working.iat[i, 14] = 0
                Axis_Working.iat[i, 15] = 0
                Axis_Working.iat[i, 16] = 0
                Axis_Working.iat[i, 17] = 0
                Axis_Working.iat[i, 18] = 0
                Axis_Working.iat[i, 19] = 0
                Axis_Working.iat[i, 20] = 0
                Axis_Working.iat[i, 21] = 0
                Axis_Working.iat[i, 22] = 0
                Axis_Working.iat[i, 23] = 0
                Axis_Working.iat[i, 24] = 1
                Axis_Working.iat[i, 25] = 0
                Axis_Working.iat[i, 26] = 0
                Axis_Working.iat[i, 27] = 0
                Axis_Working.iat[i, 28] = 0
                Axis_Working.iat[i, 29] = 0
                Axis_Working.iat[i, 30] = 0
                Axis_Working.iat[i, 31] = 0
              
                
    if(Axis_Working.iat[i, 11] == "Bullish Harami"):
                Axis_Working.iat[i, 12] = 0
                Axis_Working.iat[i, 13] = 0
                Axis_Working.iat[i, 14] = 0
                Axis_Working.iat[i, 15] = 0
                Axis_Working.iat[i, 16] = 0
                Axis_Working.iat[i, 17] = 0
                Axis_Working.iat[i, 18] = 0
                Axis_Working.iat[i, 19] = 0
                Axis_Working.iat[i, 20] = 0
                Axis_Working.iat[i, 21] = 0
                Axis_Working.iat[i, 22] = 0
                Axis_Working.iat[i, 23] = 0
                Axis_Working.iat[i, 24] = 0
                Axis_Working.iat[i, 25] = 1
                Axis_Working.iat[i, 26] = 0
                Axis_Working.iat[i, 27] = 0
                Axis_Working.iat[i, 28] = 0
                Axis_Working.iat[i, 29] = 0
                Axis_Working.iat[i, 30] = 0
                Axis_Working.iat[i, 31] = 0
          
                
    if(Axis_Working.iat[i, 11] == "Bearish Harami"):
                Axis_Working.iat[i, 12] = 0
                Axis_Working.iat[i, 13] = 0
                Axis_Working.iat[i, 14] = 0
                Axis_Working.iat[i, 15] = 0
                Axis_Working.iat[i, 16] = 0
                Axis_Working.iat[i, 17] = 0
                Axis_Working.iat[i, 18] = 0
                Axis_Working.iat[i, 19] = 0
                Axis_Working.iat[i, 20] = 0
                Axis_Working.iat[i, 21] = 0
                Axis_Working.iat[i, 22] = 0
                Axis_Working.iat[i, 23] = 0
                Axis_Working.iat[i, 24] = 0
                Axis_Working.iat[i, 25] = 0
                Axis_Working.iat[i, 26] = 1
                Axis_Working.iat[i, 27] = 0
                Axis_Working.iat[i, 28] = 0
                Axis_Working.iat[i, 29] = 0
                Axis_Working.iat[i, 30] = 0
                Axis_Working.iat[i, 31] = 0
          
    if(Axis_Working.iat[i, 11] == "Morning Star"):
                Axis_Working.iat[i, 12] = 0
                Axis_Working.iat[i, 13] = 0
                Axis_Working.iat[i, 14] = 0
                Axis_Working.iat[i, 15] = 0
                Axis_Working.iat[i, 16] = 0
                Axis_Working.iat[i, 17] = 0
                Axis_Working.iat[i, 18] = 0
                Axis_Working.iat[i, 19] = 0
                Axis_Working.iat[i, 20] = 0
                Axis_Working.iat[i, 21] = 0
                Axis_Working.iat[i, 22] = 0
                Axis_Working.iat[i, 23] = 0
                Axis_Working.iat[i, 24] = 0
                Axis_Working.iat[i, 25] = 0
                Axis_Working.iat[i, 26] = 0
                Axis_Working.iat[i, 27] = 1
                Axis_Working.iat[i, 28] = 0
                Axis_Working.iat[i, 29] = 0
                Axis_Working.iat[i, 30] = 0
                Axis_Working.iat[i, 31] = 0
            
                
    if(Axis_Working.iat[i, 11] == "Evening Star"):
                Axis_Working.iat[i, 12] = 0
                Axis_Working.iat[i, 13] = 0
                Axis_Working.iat[i, 14] = 0
                Axis_Working.iat[i, 15] = 0
                Axis_Working.iat[i, 16] = 0
                Axis_Working.iat[i, 17] = 0
                Axis_Working.iat[i, 18] = 0
                Axis_Working.iat[i, 19] = 0
                Axis_Working.iat[i, 20] = 0
                Axis_Working.iat[i, 21] = 0
                Axis_Working.iat[i, 22] = 0
                Axis_Working.iat[i, 23] = 0
                Axis_Working.iat[i, 24] = 0
                Axis_Working.iat[i, 25] = 0
                Axis_Working.iat[i, 26] = 0
                Axis_Working.iat[i, 27] = 0
                Axis_Working.iat[i, 28] = 1
                Axis_Working.iat[i, 29] = 0
                Axis_Working.iat[i, 30] = 0
                Axis_Working.iat[i, 31] = 0
             
                
    if(Axis_Working.iat[i, 11] == "Soldiers"):
                Axis_Working.iat[i, 12] = 0
                Axis_Working.iat[i, 13] = 0
                Axis_Working.iat[i, 14] = 0
                Axis_Working.iat[i, 15] = 0
                Axis_Working.iat[i, 16] = 0
                Axis_Working.iat[i, 17] = 0
                Axis_Working.iat[i, 18] = 0
                Axis_Working.iat[i, 19] = 0
                Axis_Working.iat[i, 20] = 0
                Axis_Working.iat[i, 21] = 0
                Axis_Working.iat[i, 22] = 0
                Axis_Working.iat[i, 23] = 0
                Axis_Working.iat[i, 24] = 0
                Axis_Working.iat[i, 25] = 0
                Axis_Working.iat[i, 26] = 0
                Axis_Working.iat[i, 27] = 0
                Axis_Working.iat[i, 28] = 0
                Axis_Working.iat[i, 29] = 1
                Axis_Working.iat[i, 30] = 0
                Axis_Working.iat[i, 31] = 0
         
                
    if(Axis_Working.iat[i, 11] == "Crows"):
                Axis_Working.iat[i, 12] = 0
                Axis_Working.iat[i, 13] = 0
                Axis_Working.iat[i, 14] = 0
                Axis_Working.iat[i, 15] = 0
                Axis_Working.iat[i, 16] = 0
                Axis_Working.iat[i, 17] = 0
                Axis_Working.iat[i, 18] = 0
                Axis_Working.iat[i, 19] = 0
                Axis_Working.iat[i, 20] = 0
                Axis_Working.iat[i, 21] = 0
                Axis_Working.iat[i, 22] = 0
                Axis_Working.iat[i, 23] = 0
                Axis_Working.iat[i, 24] = 0
                Axis_Working.iat[i, 25] = 0
                Axis_Working.iat[i, 26] = 0
                Axis_Working.iat[i, 27] = 0
                Axis_Working.iat[i, 28] = 0
                Axis_Working.iat[i, 29] = 0
                Axis_Working.iat[i, 30] = 1
                Axis_Working.iat[i, 31] = 0
            
                
    if(Axis_Working.iat[i, 11] == "None"):
                Axis_Working.iat[i, 12] = 0
                Axis_Working.iat[i, 13] = 0
                Axis_Working.iat[i, 14] = 0
                Axis_Working.iat[i, 15] = 0
                Axis_Working.iat[i, 16] = 0
                Axis_Working.iat[i, 17] = 0
                Axis_Working.iat[i, 18] = 0
                Axis_Working.iat[i, 19] = 0
                Axis_Working.iat[i, 20] = 0
                Axis_Working.iat[i, 21] = 0
                Axis_Working.iat[i, 22] = 0
                Axis_Working.iat[i, 23] = 0
                Axis_Working.iat[i, 24] = 0
                Axis_Working.iat[i, 25] = 0
                Axis_Working.iat[i, 26] = 0
                Axis_Working.iat[i, 27] = 0
                Axis_Working.iat[i, 28] = 0
                Axis_Working.iat[i, 29] = 0
                Axis_Working.iat[i, 30] = 0
                Axis_Working.iat[i, 31] = 1
                
                

                
#Generating Technical Indicators

rsi_7 = RSIIndicator(Axis_Working.Close, window = 7)
Axis_Working["RSI_7"] = rsi_7.rsi()
#Axis_Working["RSI_7"] = minmax_scale(Axis_Working["RSI_7"], (0,1))

rsi_14 = RSIIndicator(Axis_Working.Close, window = 14)
Axis_Working["RSI_14"] = rsi_14.rsi()  
#Axis_Working["RSI_14"] = minmax_scale(Axis_Working["RSI_14"], (0,1))

rsi_21 = RSIIndicator(Axis_Working.Close, window = 21)
Axis_Working["RSI_21"] = rsi_21.rsi()
#Axis_Working["RSI_21"] = minmax_scale(Axis_Working["RSI_21"], (0,1))

Axis_Working["MACD (26, 12)"] = macd(Axis_Working.Close, window_slow = 26, window_fast = 12)
#Axis_Working["MACD (26, 12)"] = minmax_scale(Axis_Working["MACD (26, 12)"], (0,1))

Axis_Working["MACD (44, 26)"] = macd(Axis_Working.Close, window_slow = 44, window_fast = 26)
#Axis_Working["MACD (44, 26)"] = minmax_scale(Axis_Working["MACD (44, 26)"], (0,1))

obv = OnBalanceVolumeIndicator(Axis_Working.Close, Axis_Working.Volume)
Axis_Working["OBV"] = obv.on_balance_volume()
#Axis_Working["OBV"] = minmax_scale(Axis_Working["OBV"], (0,1))

atr = AverageTrueRange(Axis_Working.High, Axis_Working.Low, Axis_Working.Close, window = 7)
Axis_Working["ATR_7"] = atr.average_true_range()
#Axis_Working["ATR_7"] = minmax_scale(Axis_Working["ATR_7"], (0,1))

atr = AverageTrueRange(Axis_Working.High, Axis_Working.Low, Axis_Working.Close, window = 14)
Axis_Working["ATR_14"] = atr.average_true_range()
#Axis_Working["ATR_14"] = minmax_scale(Axis_Working["ATR_14"], (0,1))

atr = AverageTrueRange(Axis_Working.High, Axis_Working.Low, Axis_Working.Close, window = 21)
Axis_Working["ATR_21"] = atr.average_true_range()
#Axis_Working["ATR_21"] = minmax_scale(Axis_Working["ATR_21"], (0,1))

ao = AwesomeOscillatorIndicator(Axis_Working.High, Axis_Working.Low, window1 = 5, window2 = 34, fillna = False)
Axis_Working["AO"] = ao.awesome_oscillator()
#Axis_Working["AO"] = minmax_scale(Axis_Working["AO"], (0,1))

Kama = KAMAIndicator(Axis_Working.Close, window  = 10, pow1 = 2, pow2 = 30, fillna = False)
Axis_Working["KAMA"] = Kama.kama()
#Axis_Working["KAMA"] = minmax_scale(Axis_Working["KAMA"], (0,1))

PPO = PercentagePriceOscillator(Axis_Working.Close, window_slow = 26, window_fast = 12, window_sign = 9, fillna = False)
Axis_Working["PPO"] = PPO.ppo()
#Axis_Working["PPO"] = minmax_scale(Axis_Working["PPO"], (0,1))

PPO_Signal = PercentagePriceOscillator(Axis_Working.Close, window_slow = 26, window_fast = 12, window_sign = 9, fillna = False)
Axis_Working["PPO_Signal"] = PPO_Signal.ppo_signal()
#Axis_Working["PPO_Signal"] = minmax_scale(Axis_Working["PPO_Signal"], (0,1))

PVO = PercentageVolumeOscillator(Axis_Working.Volume, window_slow = 26, window_fast = 12, window_sign = 9, fillna = False)
Axis_Working["PVO"] = PVO.pvo()
#Axis_Working["PVO"] = minmax_scale(Axis_Working["PVO"], (0,1))

PVO_Signal = PercentageVolumeOscillator(Axis_Working.Volume, window_slow = 26, window_fast = 12, window_sign = 9, fillna = False)
Axis_Working["PVO_Signal"] = PVO_Signal.pvo_signal()
#Axis_Working["PVO_Signal"] = minmax_scale(Axis_Working["PVO_Signal"], (0,1))

ROC = ROCIndicator(Axis_Working.Close, window = 7, fillna = False)
Axis_Working["ROC"] = ROC.roc()
#Axis_Working["ROC"] = minmax_scale(Axis_Working["ROC"], (0,1))

SRSI = StochRSIIndicator(Axis_Working.Close, window = 14, smooth1 = 7, smooth2 = 7, fillna = False)
Axis_Working["Stochastic RSI"] = SRSI.stochrsi()
#Axis_Working["Stochastic RSI"] = minmax_scale(Axis_Working["Stochastic RSI"], (0,1))

SRSI_d = StochRSIIndicator(Axis_Working.Close, window = 14, smooth1 = 7, smooth2 = 7, fillna = False)
Axis_Working["Stochastic RSI_d"] = SRSI_d.stochrsi_d()
#Axis_Working["Stochastic RSI_d"] = minmax_scale(Axis_Working["Stochastic RSI_d"], (0,1))

SRSI_k = StochRSIIndicator(Axis_Working.Close, window = 14, smooth1 = 7, smooth2 = 7, fillna = False)
Axis_Working["Stochastic RSI_k"] = SRSI_k.stochrsi_k()
#Axis_Working["Stochastic RSI_k"] = minmax_scale(Axis_Working["Stochastic RSI_k"], (0,1))

TSI = TSIIndicator(Axis_Working.Close, window_slow = 26, window_fast = 12, fillna = False)
Axis_Working["TSI"] = TSI.tsi()
#Axis_Working["TSI"] = minmax_scale(Axis_Working["TSI"], (0,1))

WR = WilliamsRIndicator(Axis_Working.High, Axis_Working.Low, Axis_Working.Close, lbp = 14, fillna = False)
Axis_Working["WR"] = WR.williams_r()
#Axis_Working["WR"] = minmax_scale(Axis_Working["WR"], (0,1))

adi = AccDistIndexIndicator(Axis_Working.High, Axis_Working.Low, Axis_Working.Close, Axis_Working.Volume, fillna = False)
Axis_Working["ADI"] = adi.acc_dist_index()
#Axis_Working["ADI"] = minmax_scale(Axis_Working["ADI"], (0,1))

cmi = ChaikinMoneyFlowIndicator(Axis_Working.High, Axis_Working.Low, Axis_Working.Close, Axis_Working.Volume, window = 14, fillna = False)
Axis_Working["CMI"] = cmi.chaikin_money_flow()
#Axis_Working["CMI"] = minmax_scale(Axis_Working["CMI"], (0,1))

fi = ForceIndexIndicator(Axis_Working.Close, Axis_Working.Volume, window = 14, fillna = False)
Axis_Working["FI"] = fi.force_index()
#Axis_Working["FI"] = minmax_scale(Axis_Working["FI"], (0,1))

mfi = MFIIndicator(Axis_Working.High, Axis_Working.Low, Axis_Working.Close, Axis_Working.Volume, window = 14, fillna = False)
Axis_Working["MFI"] = mfi.money_flow_index()
#Axis_Working["MFI"] = minmax_scale(Axis_Working["MFI"], (0,1))

nvi = NegativeVolumeIndexIndicator(Axis_Working.Close, Axis_Working.Volume, fillna = False)
Axis_Working["NVI"] = nvi.negative_volume_index()
#Axis_Working["NVI"] = minmax_scale(Axis_Working["NVI"], (0,1))

vpt = VolumePriceTrendIndicator(Axis_Working.Close, Axis_Working.Volume, fillna = False)
Axis_Working["VPT"] = vpt.volume_price_trend()
#Axis_Working["VPT"] = minmax_scale(Axis_Working["VPT"], (0,1))

ADX = ADXIndicator(Axis_Working.High, Axis_Working.Low, Axis_Working.Close, window = 14, fillna = False)
Axis_Working["ADX"] = ADX.adx()
#Axis_Working["ADX"] = minmax_scale(Axis_Working["ADX"], (0,1))

mass = MassIndex(Axis_Working.High, Axis_Working.Low, window_fast = 7, window_slow = 14, fillna = False)
Axis_Working["MASS INDEX"] = mass.mass_index()
#Axis_Working["MASS INDEX"] = minmax_scale(Axis_Working["MASS INDEX"], (0,1))

# Direction labels as per the previous day's close

Axis_Working["Direction"] = ""

for i in range (len(Axis_Working)):
    
    if(Axis_Working.iat[i-1, 4] < Axis_Working.iat[i, 4]):
        Axis_Working.iat[i, 61] = 1
        
    else:
        Axis_Working.iat[i, 61] = 0

# Target labels 

Axis_Working["Target"] = ""

for i in range (len(Axis_Working) - 1):
    
    Axis_Working.iat[i, 62] = Axis_Working.iat[i+1, 61]
    
# Calculating Candle Score

Axis_Working["Candle Score"] = ""

r = 0 
w = 0
s = 0

for i in range (2461):
    
    if(Axis_Working.iat[i, 11] == "Marubozu"):
       
        if((Axis_Working.iat[i, 10] == 1 and Axis_Working.iat[i+1, 61] == 1) or (Axis_Working.iat[i, 10] == 0 and Axis_Working.iat[i+1, 61] == 0)):
                r = r + 1
                
        elif((Axis_Working.iat[i, 10] == 1 and Axis_Working.iat[i+1, 61] == 0) or (Axis_Working.iat[i, 10] == 0 and Axis_Working.iat[i+1, 61] == 1)):
                w = w + 1
                
s = r/(r + w)        
               
for i in range (2461):
    
     if(Axis_Working.iat[i, 11] == "Marubozu"):
            Axis_Working.iat[i, 63] = s

r = 0 
w = 0
s = 0

for i in range (2461):
    
    if(Axis_Working.iat[i, 11] == "Hammer"):
       
        if(Axis_Working.iat[i+1, 61] == 1):
                r = r + 1
                
        elif(Axis_Working.iat[i+1, 61] == 0):
                w = w + 1
                
s = r/(r + w)        
               
for i in range (2461):
    
     if(Axis_Working.iat[i, 11] == "Hammer"):
            Axis_Working.iat[i, 63] = s
            
            
r = 0 
w = 0
s = 0

for i in range (2461):
    
    if(Axis_Working.iat[i, 11] == "Hanging Man"):
       
        if(Axis_Working.iat[i+1, 61] == 0):
                r = r + 1
                
        elif(Axis_Working.iat[i+1, 61] == 1):
                w = w + 1
                
s = r/(r + w)        
               
for i in range (2461):
    
     if(Axis_Working.iat[i, 11] == "Hanging Man"):
            Axis_Working.iat[i, 63] = s
            
            
r = 0 
w = 0
s = 0

for i in range (2461):
    
    if(Axis_Working.iat[i, 11] == "Bullish Engulfing"):
       
        if(Axis_Working.iat[i+1, 61] == 1):
                r = r + 1
                
        elif(Axis_Working.iat[i+1, 61] == 0):
                w = w + 1
                
s = r/(r + w)        
               
for i in range (2461):
    
     if(Axis_Working.iat[i, 11] == "Bullish Engulfing"):
            Axis_Working.iat[i, 63] = s
            
            
r = 0 
w = 0
s = 0

for i in range (2461):
    
    if(Axis_Working.iat[i, 11] == "Bearish Engulfing"):
       
        if(Axis_Working.iat[i+1, 61] == 0):
                r = r + 1
                
        elif(Axis_Working.iat[i+1, 61] == 1):
                w = w + 1
                
s = r/(r + w)        
               
for i in range (2461):
    
     if(Axis_Working.iat[i, 11] == "Bearish Engulfing"):
            Axis_Working.iat[i, 63] = s

r = 0 
w = 0
s = 0

for i in range (2461):
    
    if(Axis_Working.iat[i, 11] == "Soldiers"):
       
        if(Axis_Working.iat[i+1, 61] == 1):
                r = r + 1
                
        elif(Axis_Working.iat[i+1, 61] == 0):
                w = w + 1
                
s = r/(r + w)        
               
for i in range (2461):
    
     if(Axis_Working.iat[i, 11] == "Soldiers"):
            Axis_Working.iat[i, 63] = s


r = 0 
w = 0
s = 0

for i in range (2461):
    
    if(Axis_Working.iat[i, 11] == "Crows"):
       
        if(Axis_Working.iat[i+1, 61] == 0):
                r = r + 1
                
        elif(Axis_Working.iat[i+1, 61] == 1):
                w = w + 1
                
s = r/(r + w)        
               
for i in range (2461):
    
     if(Axis_Working.iat[i, 11] == "Crows"):
            Axis_Working.iat[i, 63] = s
            
r = 0 
w = 0
s = 0

for i in range (2461):
    
    if(Axis_Working.iat[i, 11] == "Morning Star"):
       
        if(Axis_Working.iat[i+1, 61] == 1):
                r = r + 1
                
        elif(Axis_Working.iat[i+1, 61] == 0):
                w = w + 1
                
s = r/(r + w)        
               
for i in range (2461):
    
     if(Axis_Working.iat[i, 11] == "Morning Star"):
            Axis_Working.iat[i, 63] = s
            
r = 0 
w = 0
s = 0

for i in range (2461):
    
    if(Axis_Working.iat[i, 11] == "Evening Star"):
       
        if(Axis_Working.iat[i+1, 61] == 0):
                r = r + 1
                
        elif(Axis_Working.iat[i+1, 61] == 1):
                w = w + 1
                
s = r/(r + w)        
               
for i in range (2461):
    
     if(Axis_Working.iat[i, 11] == "Evening Star"):
            Axis_Working.iat[i, 63] = s
            
r = 0 
w = 0
s = 0

for i in range (2461):
    
    if(Axis_Working.iat[i, 11] == "Piercing"):
       
        if(Axis_Working.iat[i+1, 61] == 1):
                r = r + 1
                
        elif(Axis_Working.iat[i+1, 61] == 0):
                w = w + 1
                
s = r/(r + w)        
               
for i in range (2461):
    
     if(Axis_Working.iat[i, 11] == "Piercing"):
            Axis_Working.iat[i, 63] = s
            
r = 0 
w = 0
s = 0

for i in range (2461):
    
    if(Axis_Working.iat[i, 11] == "Dark Cloud Cover"):
       
        if(Axis_Working.iat[i+1, 61] == 0):
                r = r + 1
                
        elif(Axis_Working.iat[i+1, 61] == 1):
                w = w + 1
                
s = r/(r + w)        
               
for i in range (2461):
    
     if(Axis_Working.iat[i, 11] == "Dark Cloud Cover"):
            Axis_Working.iat[i, 63] = s
            
r = 0 
w = 0
s = 0

for i in range (2461):
    
    if(Axis_Working.iat[i, 11] == "Bullish Harami"):
       
        if(Axis_Working.iat[i+1, 61] == 1):
                r = r + 1
                
        elif(Axis_Working.iat[i+1, 61] == 0):
                w = w + 1
                
s = r/(r + w)        
               
for i in range (2461):
    
     if(Axis_Working.iat[i, 11] == "Bullish Harami"):
            Axis_Working.iat[i, 63] = s

r = 0 
w = 0
s = 0

for i in range (2461):
    
    if(Axis_Working.iat[i, 11] == "Bearish Harami"):
       
        if(Axis_Working.iat[i+1, 61] == 0):
                r = r + 1
                
        elif(Axis_Working.iat[i+1, 61] == 1):
                w = w + 1
                
s = r/(r + w)        
               
for i in range (2461):
    
     if(Axis_Working.iat[i, 11] == "Bearish Harami"):
            Axis_Working.iat[i, 63] = s
            

r = 0 
w = 0
s = 0

for i in range (2461):
    
    if(Axis_Working.iat[i, 11] == "Inverted Hammer"):
       
        if(Axis_Working.iat[i+1, 61] == 1):
                r = r + 1
                
        elif(Axis_Working.iat[i+1, 61] == 0):
                w = w + 1
                
s = r/(r + w)        
               
for i in range (2461):
    
     if(Axis_Working.iat[i, 11] == "Inverted Hammer"):
            Axis_Working.iat[i, 63] = s
            

            
for i in range (2461):
    
    if(Axis_Working.iat[i, 11] == "None"):
        Axis_Working.iat[i, 63] = 0

        
Axis_Working["Candle Body"] = abs(Axis_Working["Candle Body"]).copy()
Axis_Working = Axis_Working.drop(['Long', 'Short', 'Date'], axis = 1)

Axis_Working = Axis_Working[43 : ]

Axis_Working_Train = Axis_Working[ : 2160]
Axis_Working_Test = Axis_Working[2180 : ]

#Axis_Working.to_csv(r'C:\Users\MOHIT.K\Desktop\Axis_Main.csv', index = False)
#Axis_Working_Train.to_csv(r'C:\Users\MOHIT.K\Desktop\Axis_Train.csv', index = False)
#Axis_Working_Test.to_csv(r'C:\Users\MOHIT.K\Desktop\Axis_Test.csv', index = False)

<ipython-input-1-4286aa9cff75>:115: RuntimeWarning: invalid value encountered in double_scalars
  if(0 <= Axis_Working.iat[i, 7]/Axis_Working.iat[i, 9] <= 0.15 and 0 <= Axis_Working.iat[i, 8]/Axis_Working.iat[i, 9] <= 0.15):
<ipython-input-1-4286aa9cff75>:120: RuntimeWarning: invalid value encountered in double_scalars
  if(0.1 < abs(Axis_Working.iat[i, 6])/Axis_Working.iat[i, 9] <= 0.3 and 0.35<=Axis_Working.iat[i, 7]/Axis_Working.iat[i, 9]<=0.45):
<ipython-input-1-4286aa9cff75>:125: RuntimeWarning: invalid value encountered in double_scalars
  if(0 <= abs(Axis_Working.iat[i, 6])/Axis_Working.iat[i, 9] <= 0.05):
/home/kaustubh/anaconda3/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/home/kaustubh/anaconda3/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


In [2]:
Axis_Working_Train

,Open,High,Low,Close,Volume,Candle Body,Upper Shadow,Lower Shadow,Range,Candle_Nature,...,CMI,FI,MFI,NVI,VPT,ADX,MASS INDEX,Direction,Target,Candle Score
43,289.540009,292.109985,287.200012,290.170013,1020425.0,0.630004,1.93997,2.34,4.90997,1,...,0.328729,1.586918e+06,87.890010,1104.471129,13953.884018,21.504248,13.700413,1,0,
44,289.000000,289.000000,281.399994,282.429993,454510.0,6.570007,0,1.03,7.60001,0,...,0.286698,9.062738e+05,83.851828,1075.010371,-9726.621867,21.407278,13.789974,0,1,0.50463
45,279.200012,289.000000,278.209991,287.880005,627395.0,8.679993,1.12,0.990021,10.79,1,...,0.287971,1.241345e+06,83.981331,1075.010371,-16.885611,20.733786,14.000841,1,0,0.50463
46,284.000000,291.559998,284.000000,285.470001,469505.0,1.470001,6.09,0,7.56,1,...,0.287637,9.249647e+05,83.782442,1066.010860,8176.265323,20.457313,14.184119,0,0,
47,286.600006,289.959991,277.600006,278.959991,518155.0,7.640015,3.35998,1.35998,12.36,0,...,0.162896,3.518769e+05,77.519222,1066.010860,-15746.770770,19.115524,14.361438,0,1,0.486301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2198,746.150024,750.250000,735.400024,737.299988,166417.0,8.850036,4.09998,1.89996,14.85,0,...,-0.204329,-6.597325e+04,50.948290,496.538783,-2325.629207,12.738317,14.401978,0,1,
2199,736.299988,742.000000,731.000000,739.799988,137613.0,3.500000,2.20001,5.29999,11,1,...,-0.127827,-1.130581e+04,43.822208,498.222422,-1507.249719,11.837927,14.268004,1,0,0
2200,742.000000,746.049988,724.950012,726.750000,161423.0,15.250000,4.04999,1.79999,21.1,0,...,-0.119584,-2.906741e+05,34.519674,498.222422,-2380.872146,11.517939,14.258757,0,0,0.486301
2201,725.049988,729.500000,716.200012,717.750000,219480.0,7.299988,4.45001,1.54999,13.3,0,...,-0.149215,-5.152936e+05,33.086850,498.222422,-5565.502013,11.911385,14.245688,0,0,0
